<a href="https://colab.research.google.com/github/YeonJaeHong/ESAA_homeworks/blob/main/10_6_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이썬 머신러닝 : 174-227
06. 피마 인디언 당뇨병 예측

북아메리카 피마 지역 원주민의 type-2 당뇨병 결과 데이터
: 당뇨 원인은 식습관과 유전으로, 피마 인디언 당뇨병 세트는 고립된 유전적 특성 때문에 연구대상으로 선정되었음

Feature

* pregnancies 임신 횟수, flucose 포도당 부하 검사 수치, bloodpressure 혈압, skinthickness 팔 삼두근 뒤똑의 피하지방 측정값, insulin 혈청 인슐린, bmi 체질량지수, diabetespredigreefunction 당뇨 내력 가중치 값, age 나이, outcome 클래스 결정값

In [ ]:
#기초 코드
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
diabetes_data = pd.read_csv('/content/diabetes.csv')
print(diabetes_data['Outcome'].value_counts())
diabetes_data.head(3)


FileNotFoundError: ignored

In [ ]:
diabetes_data.info()

feature 타입과 null 개수 : null 없고 피처 타입 모두 숫자: 별다른 전처리를 해줄 필요는 없다

로지스틱 회귀 이용해 예측 모델 생성

-> 데이터 세트를 피처 데이터 세트와 클래스 데이터 세트로 나누고 학습 데이터 세트와 테스트 데이터 세트로 분리
-> 유틸리티 함수들 사용 후, 성능 평가 지표를 출력하고 재현율 곡선을 시각화하겠음

In [ ]:
#get_clf_eval 함수
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.preprocessing import Binarizer
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    #thresholds list 객체 내의 값을 iteration 하면서 평가 수행
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임계값: ', custom_threshold)
        get_clf_eval(y_test, custom_predict)

In [ ]:
def precision_recall_curve_plot(y_test, pred_proba_c1):
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)

    # x축을 threshold 값, y축을 정밀도, 재현율로 그리기
    plt.figure(figsize=(8,6))
    thresholds_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0: thresholds_boundary], linestyle= '--', label='precision')
    plt.plot(thresholds, recalls[0: thresholds_boundary], label='recall')

    # threshold의 값 X축의 scale을 0.1 단위로 변경
    stard, end = plt.xlim()
    plt.xticks(np.round(np.arange(stard, end, 0.1), 2))

    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value')
    plt.ylabel('Precision and Recall value')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def get_clf_eval(y_test, pred=None,pred_prova=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    #ROC_AUC 추가
    roc_score = roc_auc_score(y_test, pred_proba)
    print('오차행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('\n정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nF1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1,roc_score))




In [ ]:
#피처 데이터 세트 x, 레이블 데이터 세트 y 추출
#맨 끝이 Outcome 칼럼으로 레이블 값임. 칼럼 위치 -1을 이용해 추출
X = diabetes_data.iloc[:, :-1]
y = diabetes_data.iloc[:,-1] #outcome
print(X)
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156, stratify=y)

#로지스틱 회귀로 학습, 예측 및 평가 수행
Ir_clf = LogisticRegression(solver='liblinear')
Ir_clf.fit(X_train, y_train)
pred = Ir_clf.predict(X_test)
pred_proba = Ir_clf.predict_proba(X_test)[:,1]

get_clf_eval(y_test, pred, pred_proba)


In [ ]:
##### 추가적으로 iloc 에 대한 설명

#data.iloc[0] # data의 첫번째 행만
#data.iloc[1] # 두번째 행만
#data.iloc[-1] # 마지막 행만

#### Columns:
#data.iloc[:,0] # 첫번째 열만
#data.iloc[:,1] # 두번째 열만
#data.iloc[:,-1] # 마지막 열만

정밀도 재현율 곡선을 보고 임곗값별 정밀도와 재현율 값의 변화를 확인: 이를 위해서 precision_recall_curve_plot() 함수를 이용

In [ ]:
pred_proba_c1 = Ir_clf.predict_proba(X_test)[:,1]
precision_recall_curve_plot(y_test, pred_proba_c1)

재현율 곡선을 보면 임곗값을 0.42 정도로 맞추면 정밀도와 재현율이 균형을 맞을 것 같음  조작하기 전 원보 ㄴ데이터의 값을 점검

In [ ]:
diabetes_data.describe()

min() 값이 0 이 되어있는 피처가 많은데, glucose 피처의 히스토그램을 확인해보면 0 값이 일정 수준 존재하는 것을 확인 할 수 있다.

In [ ]:
plt.hist(diabetes_data['Glucose'],bins=10)

In [ ]:
# min()값이 0으로 돼있는 피처에 대해 0 값의 건수 및 전체 데이터 건수 대비 몇 퍼센트의 비율로 존재하는지 확인
#0값을 검사할 피처명 리스트
zero_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

#전체 데이터 건수
total_count = diabetes_data['Glucose'].count()

# 피처별로 반복하면서 데이터 값이 0인 데이터 건수를 추출하고, 퍼센트 계산
for feature in zero_features:
    zero_count = diabetes_data[diabetes_data[feature]==0][feature].count() #행은 값이 0인것, 열은 feature 로 둬야지 갯수 셀수있
    print('{0}0 건수는 {1}, 퍼센트는{2:.2f}%'.format(feature, zero_count, 100*zero_count/total_count))



SkinThickness, Insulin의 0값은 상당히 높은 비율로 존재.
 하지만 전체 dataset 수가 많지 않은 상황이라 이 data들을 일괄적으로 삭제할 경우 학습을 효과적으로 수행하기 어려울것 같아 위 피처의 0 값을 평균값으로 대체

In [ ]:
#zero_features 리스트 내부에 저장된 개별 피처들에 대해서 0값을 평균 값으로 대체
mean_zero_features = diabetes_data[zero_features].mean()
diabetes_data[zero_features]=diabetes_data[zero_features].replace(0,mean_zero_features)

로지스틱 회귀의 경우 일반적으로 숫자 데이터에 스케일을 적용하는것이 좋기 떄문에 스케일링 이후 학습/테스트 데이터로 나누고 로지스틱 회귀를 적용해서 성능 평가 지표를 확인하는 것이 좋음

In [ ]:
X = diabetes_data.iloc[:,:-1]
y = diabetes_data.iloc[:,-1]

#standardScaler 클래스를 이용해 피처 데이터 세트에 일괄적으로 스케일링 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=156, stratify=y)

#로지스틱 회귀로 학습, 예측 및 평가 수행
Ir_clf = LogisticRegression()
Ir_clf.fit(X_train, y_train)
pred = Ir_clf.predict(X_test)
pred_proba = Ir_clf.predict_proba(X_test)[:,1]

get_clf_eval(y_test, pred, pred_proba)

분류 결정 임곗값을 0.3에서 0.5까지 0.03 씩 변화시키면서 성능 수치가 어느 정도 개선되는지 확인

In [ ]:
from sklearn.preprocessing import Binarizer

def get_eval_by_threshold(y_test , pred_proba_c1, thresholds):
    # thresholds 리스트 객체내의 값을 차례로 iteration하면서 Evaluation 수행.
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임곗값:',custom_threshold)
        print('-'*80)
        get_clf_eval(y_test , custom_predict, pred_proba_c1)
        print('='*80)

thresholds = [0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.50]
pred_proba = lr_clf.predict_proba(X_test)
get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

In [ ]:
#임곗값을 0.48로 설정한 Binarizer 생성
binarizer = Binarizer(threshold=0.48)

#위에서 구한 Ir_clf.predict_proba() 예측 확률 array 에서 1에 해당하는 칼럼값을 Binarizer 변환
pred_th_048 = binarizer.fit_transform(pred_proba[:,1].reshape(-1,1))
#get_clf_eval(y_test, pred_th_048, pred_proba[:,1])

CH.04  분류


01. 분류의 개요
지도학습: 레이블, 즉 명시적 정답이 있는 데이터가 주어진 상태에서 학습하는 머신러닝 방식



*   분류: 가장 대표적인 지도학습의 유형 학습 데이터로 주어진 데이터의 피처와 레이블 값을 머신러닝 알고리즘으로 학습해 모델 생성, 생성된 모델에 새로운 데이터 값이 주어졌을 때 미지의 레이블 값을 예측(패턴 인지후)


-앙상블: 서로 다른/또는 같은 알고리즘 단순히 결합 형태도 있으나 일반적으로 배깅(대표적으로, 랜덤 포레스트), 부스팅(대표적으로, 그래디언트 부스팅)으로 나뉜다.
; 그래디언트 부스팅의 경우 뛰어난 예측성능을 가지지만 수행시간이 길어 최적화 모델 튜닝이 어렵다는 단점이 있었지만, 그것을 보완한 것이 xgboost, lightbgm 이 된다.


 02. 결정트리

*   결정트리:  데이터에 있는 규칙을 학습을 통해 자동으로 찾아내 트리 기반의 분류 규칙 만들기(if/else 기반) (스무고개)
사전 가공의 영향을 가장 적게 받고 복잡한 규칙구조를 가져야지 예측성능이 향상되지만 과적합의 위험이 있다.

-> 규칙노드는 규칙 조건, 리프 노드는 결정된 클래스값, 새로운 규칙 조건마다 서브 트리 생성 -> 트리 깊이 깊어질수록 과적합 발생-> 결정 트리 에측 성능 저하

-결정노드: 정보 균일도가 높은 데이터 세트를 먼저 선택할 수 있도록 규칙 조건을 만든다.

-균일도 측정 방법: 엔트로피를 이용한 정보 이득 지수(1-엔트로피 지수)와 지니계수

1) 데이터 집합의 모든 아이템이 같은 분류에 속하는 지 확인
 2.1) 리프 노드로 만들어서 분류 결정
 2.2) 데이터를 분할하는데 가장 좋은 속성과 분할 기준 찾기
3) 해당 속성과 분할 기준으로 데이터 분할해 branch 노드 생성
4) recursive하게 모든 데이터 집합의 분류가 결정될 때까지 수행



-결정트리 모델 장점: '균일도라는 룰을 기반'으로 쉽고, 직관적, 피터의 스케일링이나 정규화 등 사전 가공 영향도가 크지 않음 그래서 정보의 균일도만 신경 쓰면 되므로 피처의 스케일링이나 정규화 과정은 필요가 없음

-결정트리 모델의 단점: 과적합으로 정확도도 떨어짐(정확도 높이기 위해 조건 계속 추가하면서 깊이 계속 커지고 결국 복잡한 학습 모델되면 에측 성능 떨어짐: 사전에 크기 제한하는 것이 성능 튜닝에 오히려 도움)

-결정 트리 파라미터: min_samples_split, min_samples_leaf, max_features, max_depth, max_leaf_nodes

-결정 트리 모델 시각화: Graphviz 패키지 사용, export_graphviz()는 함수 인자로 학습완료된 estimator, 피처 이름 리스트, 레이블 이름 리스트 입력하면 실제 트리형태로 시각화

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#DecisionTreeClassifier 생성
dt_clf = DecisionTreeClassifier(random_state = 156)

#붓꽃 데이터를 로딩, 학습 테스트 데이터 세트로 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size = 0.2, random_state=11)

#DecisionTreeClassifier 학습
dt_clf.fit(X_train, y_train)

사이킷런의 트리 모듈은 Graphiz 를 이용하기 위해서 export_graphiz()함수를 제공

In [ ]:
from sklearn.tree import export_graphviz

#export_graphiz()의 호출 결과로 out_file로 지정된 tree.dot 파일을 생성함
export_graphviz(dt_clf,out_file='tree.dot',class_names=iris_data.target_names,feature_names=iris_data.feature_names,impurity=True,filled=True)

In [ ]:
import graphviz
#위에서 생성된 tree.dot 파일을 Graphviz가 읽어서 주피터 노트북상에서 시각화
with open("tree.dot") as f:
  dot_graph = f.read()
graphviz.Source(dot_graph)
### samples = 120; 전체 데이터 120개
### value = [41, 40, 39] 각각 레이블별로 41, 40, 39개로 구성.
### 하위노드를 가질 경우 class=setosa의 개수가 41개로 가장 많다

In [ ]:
import seaborn as sns
import numpy as np
%matplotlib inline

# feature importance 추출
print("Feature importances:{0}".format(np.round(dt_clf.feature_importances_,3)))

#feature별 importance 매핑
for name, value in zip(iris_data.feature_names, dt_clf.feature_importances_):
  print('{0} : {1:.3f}'.format(name, value))

#feature importance를 column별로 시각화하기
sns.barplot(x=dt_clf.feature_importances_, y=iris_data.feature_names)


결정 트리 과적합 Overfitting


*  make_classification() 분류를 위한 테스트용 데이터를 만들 수 있음 (호출 시 반환되는 객체는 피처 데이터 세트와 클래스 레이블 데이터 세트이다)

In [ ]:

from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
%matplotlib inline

plt.title("3 Class values with 2 Features Sample data creation")

#2차원 시각화를 위해서 피처는 2개, 클래스는 3가지 유형의 분류 샘플 데이터 생성
X_features, y_labels = make_classification(n_features = 2, n_redundant=0, n_informative=2, n_classes=3, n_clusters_per_class=1, random_state=0)

#그래프 형태로 2개의 피처로 2차원 좌표 시각화, 각 클래스 값은 다른 색깔로 표시
plt.scatter(X_features[:, 0], X_features[:, 1], marker='o', c=y_labels, s=25, edgecolor='k')

In [ ]:
# 결정 기준 경계 시각화
from sklearn.tree import DecisionTreeClassifier


def visualize_boundary(model, X, y):
    fig,ax = plt.subplots()

    # 학습 데이타 scatter plot으로 나타내기
    ax.scatter(X[:, 0], X[:, 1], c=y, s=25, cmap='rainbow', edgecolor='k',
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim_start , xlim_end = ax.get_xlim()
    ylim_start , ylim_end = ax.get_ylim()

    # 호출 파라미터로 들어온 training 데이타로 model 학습 .
    model.fit(X, y)
    # meshgrid 형태인 모든 좌표값으로 예측 수행.
    xx, yy = np.meshgrid(np.linspace(xlim_start,xlim_end, num=200),np.linspace(ylim_start,ylim_end, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # contourf() 를 이용하여 class boundary 를 visualization 수행.
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap='rainbow', clim=(y.min(), y.max()),
                           zorder=1)

# 특정한 트리 생성 제약 없는 결정 트리의 학습가 결정 경계 시각화
dt_clf = DecisionTreeClassifier().fit(X_features, y_labels)
visualize_boundary(dt_clf, X_features, y_labels)

In [ ]:
# min_samples_leaf = 6 으로 트리 생성 조건을 제약한 결정 경계 시각화

dt_clf = DecisionTreeClassifier(min_samples_leaf=6).fit(X_features, y_labels)
visualize_boundary(dt_clf, X_features, y_labels)

결정 트리 실습 - 사용자 행동 인식 데이터 세트: 수집된 피처 세트 기반 결정 트리 이용해 어떤 동작인지 예측

features.txt 파일

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

feature_name_df = pd.read_csv('/content/features.txt', sep='\s+', header=None, names=['column_index', 'coulmn_name'])

feature_name = feature_name_df.iloc[:,1].values.tolist()
print(feature_name_df.info())
print('전체 피처명에서 10개만 추출', feature_name[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   column_index  561 non-null    int64 
 1   coulmn_name   561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB
None
전체 피처명에서 10개만 추출 ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


In [ ]:
# 중복된 피처명에 대해서 원본 피처명에 _1 또는 _2를 추가로 부여해 변경한뒤에 dataframe에 로드한다
feature_dup_df = feature_name_df.groupby('coulmn_name').count()
print(feature_dup_df[feature_dup_df['column_index'] > 1].count())
feature_dup_df[feature_dup_df['column_index'] > 1].head()

column_index    42
dtype: int64


,column_index
coulmn_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3


In [ ]:
def get_new_feature_name_df(old_feature_name_df):
  feature_dup_df= pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt'])
  feature_dup_df= feature_dup_df.reset_index()
  new_feature_name_df= pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
  new_feature_name_df['column_name'] = new_feature_name_df[['coulmn_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] > 0 else x[0], axis=1)

  new_feature_name_df= new_feature_name_df.drop(['index'],axis=1)
  return new_feature_name_df


In [ ]:
import pandas as pd

def get_human_dataset():
  # 각 데이터 파일은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
  feature_name_df = pd.read_csv('/content/features.txt', sep='\s+',
                                header=None, names=['column_index', 'column_name'])

  # 중복된 피처를 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame 생성
  new_feature_name_df = get_new_feature_name_df(feature_name_df)

  # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
  feature_name = new_feature_name_df.iloc[:,1].values.tolist()

  # 학습 피처 데이터 세트와 테스트 피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name 적용
  X_train = pd.read_csv('/content/X_train.txt', sep='\s+', names=feature_name)
  X_test = pd.read_csv('/content/X_test.txt', sep='\s+', names=feature_name)

  # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩. 칼럼명은 action으로 부여
  y_train = pd.read_csv('/content/y_train.txt', sep='\s+', header=None, names=['action'])
  y_test = pd.read_csv('/content/y_test.txt', sep='\s+', header=None, names=['action'])

  # 로드된 학습/테스트용 DataFrame을 모두 반환
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_human_dataset()


KeyError: ignored

In [ ]:
print(y_train['action'].value_counts())

NameError: ignored


03. 앙상블 학습
>> 앙상블 학습 개요

*   여러개의 분류기(Classifier)를 생성하고 그 예측을
결합함으로써 보다 정확한 최종 예측을 도출하는 기법

*  보팅, 배깅, 부스팅
>> 보팅 유형 - 하드 보팅(Hard Voting)과 소프트 보팅(Soft voting)
하드 보팅 : 다수결 원칙, 예측한 결괏값들중 다수의 분류가 결정한 예측값을 최종 보팅 결괏값으로 선정
소프트 보팅 : 분류기들의 레이블 값 결정 확률을 모두 더하고 이를 평균해서 가장 확률이 높은 레이블 값을 최종 보팅 결괏값으로 선정

In [53]:

import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


voting classifier 클래스는 주요 생성 인자로 estimators와 voting 값을 입력 받음
> estimators는 리스트 값으로 보팅에 사용될 여러개의 classifier 객체들을 튜플 형식으로 입력 받으며 voting은 hard 시 하드 보팅, soft 시 소프트 보팅 방식을 적용하라는 의미

In [55]:
# 개별 모델은 로지스틱 회귀와 KNN임
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=156)

# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

# 개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
  classifier.fit(X_train, y_train)
  pred = classifier.predict(X_test)
  class_name = classifier.__class__.__name__
  print('{0} 정확도 : {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

Voting 분류기 정확도: 0.9474
LogisticRegression 정확도 : 0.9386
KNeighborsClassifier 정확도 : 0.9386


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

앙상블 방법은 전반적으로 다른 단일 ML 알고리즘보다 뛰어난 예측 성능을 가지는 경우가 많다 (편향-분산 트레이드오프의 효과를 극대화 할 수 있다)

04. 랜덤 포레스트
> 랜덤 포레스트의 개요 및 실습
> 같은 알고리즘으로 여러개의 분류기를 만들어서 보팅으로 최종 결정하는 알고리즘
>> 결정 트리 사용 -> 여러개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링해서 개별적으로 학습을 수행한뒤 최종적으로 모든 분류기가 보팅을 통해 예측 결정을 하게 된

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 결정 트리에서 사용한 get_human_dataset()을 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

# 랜덤 포레스트 학습 및 별도의 테스트 세트로 예측 성능 평가
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy))

KeyError: ignored

랜덤 포레스트의 하이퍼 파라미터 및 튜닝
>

*   n_estimators : 결정 트리의 개수 지정
*   max_features




GridsearchCV를 사용 하여 하이퍼 파라미터를 튜닝 할 수 있다.


In [58]:
# 현재 이 코드에서는 n_estimators는 100으로, CV는 2로 설정해서 최적 하이퍼 파라미터를 구한다.
#n_jobs=-1 파라미터를 추가하면 모든 CPU 코어를 이용해서 학습 할 수 있다.
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[100], 'max_depth':[6,8,10,12], 'min_samples_leaf':[8,12,18], 'min_samples_split':[8,16,20]}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최고 평균 정확도 수치: {0:.4f}'.format(grid_cv.best_score_))
print('최적 하이퍼 파라미터:', grid_cv.best_params_)


최고 평균 정확도 수치: 0.9451
최적 하이퍼 파라미터: {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}


In [59]:
rf_clf1= RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=8,min_samples_split=8,random_state=0)
rf_clf1.fit(X_train,y_train)
pred=rf_clf1.predict(X_test)
print('예측정확도:{0:4f}'.format(accuracy_score(y_test,pred)))

예측정확도:0.938596


In [60]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=X_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20, y=ftr_top20.index)
plt.show()


AttributeError: ignored

05. GBM
>부스팅 알고리즘은 여러 개의 약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식
GradientBoostingClassifier 클래스 제공

In [61]:

from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행 시간: {0:.1f} 초'.format(time.time() - start_time))

KeyError: ignored

GBM의 하이퍼 파라미터 및 튜닝

> loss, learning_rate (0~1사이 값, n_estimators와 상호 보완적으로 조합하고, learning_rate을 작게하고, n_estimators를 크게 하면 더이상 성능이 좋아지지 않는 한계점까지는 예측 성능이 조금씩 좋아질 수 있음), n_estimators (기본값 100, 개수가 많을수록 예측 성능이 일정 수준까지는 좋아질 수 있음). subsample

In [62]:

from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[100,500],'learning_rate':[0.05,0.1]}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 평균 정확도 수치: {0:.4f}'.format(grid_cv.best_score_))


# GridSearchCV를 이용해 최적으로 학습된 estimator로 예측 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))

NameError: ignored